# Notebook for understanding the data and relationship

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [3]:
sql('select * from patients limit 5')

,id,birthdate,marital,race,ethnicity,gender,city,state,county,expenses,coverage
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,m,white,hispanic,m,chicopee,massachusetts,hampden county,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,m,white,nonhispanic,m,somerville,massachusetts,middlesex county,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,m,white,nonhispanic,m,chicopee,massachusetts,hampden county,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,m,white,nonhispanic,f,lowell,massachusetts,middlesex county,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,u,white,nonhispanic,m,boston,massachusetts,suffolk county,598763.07,3772.20


In [7]:
def describe(table_name):
    display(sql(f'describe {table_name}').iloc[:,:2])

In [8]:
describe('patients')

,Field,Type
0,id,text
1,birthdate,date
2,marital,text
3,race,text
4,ethnicity,text
5,gender,text
6,city,text
7,state,text
8,county,text
9,expenses,double


In [20]:
sql('select count(*) as row_counts from patients')

,row_counts
0,1171


In [21]:
sql('select count(distinct id) as total_uniqe_id from patients')

,total_uniqe_id
0,1171


In [13]:
sql('select min(birthdate) as `min`, max(birthdate) as `max` from patients')

,min,max
0,1909-12-22,2020-03-22


In [16]:
select('encounters')

,id,start,stop,patient,organization,provider,payer,class,description,basic_cost,total_cost,payer_coverage,reason
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,encounter for symptom,129.16,129.16,54.16,acute bronchitis (disorder)
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23 17:45:28,2012-01-23 18:00:28,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,wellness,general examination of patient (procedure),129.16,129.16,129.16,unknown
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01 15:02:18,2001-05-01 15:17:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,encounter for symptom,129.16,129.16,0.00,sinusitis (disorder)
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28 15:02:18,2011-07-28 15:17:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,general examination of patient (procedure),129.16,129.16,0.00,unknown
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27 12:58:08,2010-07-27 13:28:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,wellness,general examination of patient (procedure),129.16,129.16,129.16,unknown


In [22]:
sql('select count(id) as total_encounters from encounters')

,total_encounters
0,53346


In [18]:
sql('select count(distinct id) as total_encounters from encounters')

,total_encounters
0,53346


In [23]:
describe('encounters')

,Field,Type
0,id,text
1,start,datetime
2,stop,datetime
3,patient,text
4,organization,text
5,provider,text
6,payer,text
7,class,text
8,description,text
9,basic_cost,double


In [30]:
query = '''select 
        count(distinct patient) as unique_patients,
        count(distinct organization) as unique_organizations,
        count(distinct provider) as uniqe_proviers
 from encounters'''

sql(query)

,unique_patients,unique_organizations,uniqe_proviers
0,1171,1103,1104


In [28]:
tables = ['patients','organizations','providers']

for table in tables:
    display(sql(f'select count(distinct id) as total_{table}_id from {table}'))

,total_patients_id
0,1171


,total_organizations_id
0,1119


,total_providers_id
0,5855


In [31]:
select('observations')

,date,patient,encounter,description,value,units,type
0,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,body height,193.3,cm,numeric
1,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,pain severity - 0-10 verbal numeric rating [sc...,2.0,{score},numeric
2,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,body weight,87.8,kg,numeric
3,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,body mass index,23.5,kg/m2,numeric
4,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,diastolic blood pressure,82.0,mm[hg],numeric


In [39]:
sql('select count(*) as total_rows from observations')

,total_rows
0,299697


In [35]:
sql('select count(distinct encounter) as total_unique_encounters from observations')

,total_unique_encounters
0,20030


In [37]:
select('careplans')

,id,start,stop,patient,encounter,description,reason
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,respiratory therapy,acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,respiratory therapy,acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,respiratory therapy,acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,respiratory therapy,acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,wound care,laceration of foot


In [38]:
vals = ['id','patient','encounter']
for val in vals:
    query = f'''
        select count(distinct {val}) as unique_{val} from careplans
    '''
    display(sql(query))

,unique_id
0,3483


,unique_patient
0,1054


,unique_encounter
0,3472


In [40]:
sql('select count(*) as total_rows from careplans')

,total_rows
0,3483


In [41]:
select('conditions')

,start,stop,patient,encounter,description
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,acute viral pharyngitis (disorder)


In [44]:
cols = ['patient','encounter']
for col in cols:
    query = f'''
            select count(distinct {col}) as total_unique_{col} from conditions;
        '''
    display(sql(query))

,total_unique_patient
0,1152


,total_unique_encounter
0,7650


In [45]:
sql('select count(*) as total_row_count from conditions')

,total_row_count
0,8376


In [46]:
select('medications')

,start,stop,patient,payer,encounter,medicine_code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,unknown
1,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,unknown
2,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,unknown
3,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [48]:
sql('select count(*) as total_row_count from medications')

,total_row_count
0,42989


In [49]:
sql('select count(distinct encounter) as total_unique_encounters from medications')

,total_unique_encounters
0,27269


In [47]:
select('procedures')

,date,patient,encounter,description,base_cost,reason
0,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,insertion of subcutaneous contraceptive,14896.56,unknow
1,2010-07-27t12:58:08z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,medication reconciliation (procedure),726.51,unknow
2,2010-11-20t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,medication reconciliation (procedure),788.50,unknow
3,2011-02-07t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,throat culture (procedure),2070.44,acute viral pharyngitis (disorder)
4,2011-04-19t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,throat culture (procedure),2479.39,acute viral pharyngitis (disorder)


In [57]:
sql('select count(distinct encounter) as total_unique_encounters from procedures')

,total_unique_encounters
0,21072


`SO THE MAJOR UNDERSTANDING IS -> ENCOUNTERS IS THE MAIN TABLE WITH PATIENTS WHO IS VISITING, AND OTHER TABLE ARE CONNECTED TO ENCOUNTERS VIA PRIMARY KEYS OF THAT TABLE WHICH ARE PATIENT ID AND ENCOUNTER ID`